# Classify Movie Reviews using Embeddings

In [1]:
!pip install tensorflow==2.0.0.beta0

In [2]:
#this version of numpy is required to avoid an error related to numpy defaulting to not allowing pickle files
!pip install numpy==1.16.2

In [0]:
import tensorflow as tf
from tensorflow.keras import datasets, preprocessing, models, layers
import numpy as np

## Import IMDB Reviews

In [0]:
imdb = datasets.imdb

#Let's work with a dictionary of 20,000 words
NUM_WORDS = 20000

In [0]:
#load IMDB dataset as lists of integers
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=NUM_WORDS, )

## Get Word Index To See Words

In [0]:
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k: (v+3) for k, v in word_index.items()}
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2
word_index['<UNUSED>'] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

## Inspect First Review

In [7]:
print(' '.join([reverse_word_index[i] for i in train_data[0]]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be p

## Was this a positive review?

In [8]:
print(train_labels[0])

1


## Note: Reviews have variable length

In [9]:
print('Len 0: {}, Len 1: {}, Len 2: {}'.format(len(train_data[0]), len(train_data[1]), len(train_data[2])))

Len 0: 218, Len 1: 189, Len 2: 141


Variable lenght is fixed by truncating after a certain number of words. For reviews that are less than the number of words we are cutting off, we pad.

In [0]:
LEN_WORDS = 300
train_data = preprocessing.sequence.pad_sequences(train_data, maxlen=LEN_WORDS)
test_data = preprocessing.sequence.pad_sequences(test_data, maxlen=LEN_WORDS)

In [11]:
print('Len 0: {}, Len 1: {}, Len 2: {}'.format(len(train_data[0]), len(train_data[1]), len(train_data[2])))

Len 0: 300, Len 1: 300, Len 2: 300


In [12]:
print(train_data.shape)

(25000, 300)


## Sequential Model with Dense Layers

In [13]:
dense_model = models.Sequential([
    layers.Dense(300, input_shape=(300,), activation='relu'),
    layers.Dense(300, activation='relu'),
    layers.Dense(300, activation='relu'),
    layers.Dense(300, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
print(dense_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               90300     
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_2 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_3 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 301       
Total params: 361,501
Trainable params: 361,501
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
dense_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
dense_model.fit(train_data, train_labels, epochs=10)

W0612 10:51:50.880932 139758369310592 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 4s 150us/sample - loss: 7.6590 - accuracy: 0.5005
Epoch 2/10
25000/25000 [==============================] - 3s 124us/sample - loss: 7.6666 - accuracy: 0.5000
Epoch 3/10
25000/25000 [==============================] - 3s 123us/sample - loss: 7.6666 - accuracy: 0.5000
Epoch 4/10
25000/25000 [==============================] - 3s 123us/sample - loss: 7.6666 - accuracy: 0.5000
Epoch 5/10
25000/25000 [==============================] - 3s 124us/sample - loss: 7.6666 - accuracy: 0.5000
Epoch 6/10
25000/25000 [==============================] - 3s 124us/sample - loss: 7.6666 - accuracy: 0.5000
Epoch 7/10
25000/25000 [==============================] - 3s 123us/sample - loss: 7.6666 - accuracy: 0.5000
Epoch 8/10
25000/25000 [==============================] - 3s 124us/sample - loss: 7.6666 - accuracy: 0.5000
Epoch 9/10
25000/25000 [==============================] - 3s 127us/sample - loss: 7.6666 - accuracy: 0.5000
Epoch

## Introduce Embeddings

In [15]:
DIMENSION = 16

e_model = models.Sequential([
    layers.Embedding(NUM_WORDS, DIMENSION, input_length=LEN_WORDS),
    layers.GlobalAveragePooling1D(),
    layers.Dense(1, activation='sigmoid')
])
print(e_model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 16)           320000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 320,017
Trainable params: 320,017
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
e_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
e_model.fit(train_data, train_labels, epochs=10)

Train on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 4s 162us/sample - loss: 0.6408 - accuracy: 0.7414
Epoch 2/10
25000/25000 [==============================] - 4s 155us/sample - loss: 0.4794 - accuracy: 0.8372
Epoch 3/10
25000/25000 [==============================] - 4s 158us/sample - loss: 0.3703 - accuracy: 0.8763
Epoch 4/10
25000/25000 [==============================] - 4s 163us/sample - loss: 0.3089 - accuracy: 0.8924
Epoch 5/10
25000/25000 [==============================] - 4s 163us/sample - loss: 0.2695 - accuracy: 0.9054
Epoch 6/10
25000/25000 [==============================] - 4s 162us/sample - loss: 0.2404 - accuracy: 0.9173
Epoch 7/10
25000/25000 [==============================] - 4s 160us/sample - loss: 0.2178 - accuracy: 0.9242
Epoch 8/10
25000/25000 [==============================] - 4s 167us/sample - loss: 0.1981 - accuracy: 0.9323
Epoch 9/10
25000/25000 [==============================] - 4s 173us/sample - loss: 0.1810 - accuracy: 0.9386
Epoch